In [1]:
from builtins import range
import numpy as np
from random import shuffle
from sklearn.model_selection import train_test_split
from past.builtins import xrange
import pandas as pd

In [2]:
#Loading training data features
trainX = np.loadtxt('train_hog_features.csv', delimiter=',')

In [3]:
#Loading test data features
testX = np.loadtxt('test_hog_features.csv', delimiter=',')

In [4]:
X_test = testX[:,0:-1]
y_test = testX[:,-1].astype(np.int32)

In [5]:
# Dividing train data into train and validation (random)
X_train, X_val, y_train, y_val = train_test_split(trainX[:,0:-1], trainX[:,-1].astype(np.int32), test_size=0.2)


In [6]:
y_train = np.reshape(y_train,(y_train.shape[0]))
y_val = np.reshape(y_val,(y_val.shape[0]))
y_test = np.reshape(y_test,(y_test.shape[0]))

In [7]:
# Adding bias term
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])

In [8]:
X_train.shape,X_val.shape,X_test.shape

((48000, 1297), (12000, 1297), (10000, 1297))

In [9]:
def svm_loss(W, X, y, reg):
    loss = 0.0
    dW = np.zeros(W.shape) # initialize the gradient as zero

    N = X.shape[0]
    D = X.shape[1]
    C = W.shape[1]
    
    #compute scores
    scores = X.dot(W) # N x C
    
    # Record the score of the example's correct class
    correct_class_idx = tuple([range(scores.shape[0]), y])
    correct_class_scores = scores[correct_class_idx]
    
    
    # Compute for the margin by getting the max between 0 and the computed expression
    losses = scores - np.reshape(correct_class_scores,(scores.shape[0],1)) + 1
    losses[correct_class_idx] = 0
    losses = losses.clip(min=0)
    loss = np.sum(losses)
    
    # This mask can flag the examples in which their margin is greater than 0
    dscores = np.zeros((N,C))
    dscores[losses > 0] = 1
    
    d_correct_score = - np.sum(dscores, axis=1)
    dscores[correct_class_idx] = d_correct_score
    dW = X.T.dot(dscores)
    
    #Average
    loss /= N
    dW /= N
    
    #Regulariztion
    loss += reg * np.sum(W * W)
    dW += reg * 2 * W

    return loss,dW

In [17]:
print(dw.shape)

(1297, 10)


In [10]:
def train(X, y, learning_rate=1e-3, reg=1e-5, num_iters=100,
              batch_size=86, show_loss=False):
        num_train, dim = X.shape
        num_classes = np.max(y) + 1 
        
        W = 0.001 * np.random.randn(dim, num_classes)

        # Run stochastic gradient descent to optimize W
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None

            indices = np.random.choice(num_train, size=batch_size)
            X_batch = X[indices]
            y_batch = y[indices]

            # evaluate loss and gradient
            loss, grad = svm_loss(W,X_batch, y_batch, reg)
            loss_history.append(loss)
            
            W -= learning_rate * grad


            if show_loss and it % 100 == 0:
                print('iteration %d / %d: loss %f' % (it, num_iters, loss))

        return loss_history,W

In [11]:
def predict(X,W):
    scores = X.dot(W)
    y_pred = np.argmax(scores, axis=1)
    
    return y_pred

In [113]:
loss_hist,W = train(X_train, y_train, learning_rate=0.1, reg=0.0001,
                      num_iters=3000, show_loss=True)

iteration 0 / 3000: loss 8.997354
iteration 100 / 3000: loss 0.560957
iteration 200 / 3000: loss 0.721955
iteration 300 / 3000: loss 0.669788
iteration 400 / 3000: loss 0.460385
iteration 500 / 3000: loss 0.496999
iteration 600 / 3000: loss 0.499534
iteration 700 / 3000: loss 0.408102
iteration 800 / 3000: loss 0.700294
iteration 900 / 3000: loss 0.331767
iteration 1000 / 3000: loss 0.408700
iteration 1100 / 3000: loss 0.496536
iteration 1200 / 3000: loss 0.345201
iteration 1300 / 3000: loss 0.202409
iteration 1400 / 3000: loss 0.214422
iteration 1500 / 3000: loss 0.223331
iteration 1600 / 3000: loss 0.372012
iteration 1700 / 3000: loss 0.577993
iteration 1800 / 3000: loss 0.572366
iteration 1900 / 3000: loss 0.342600
iteration 2000 / 3000: loss 0.583307
iteration 2100 / 3000: loss 0.355529
iteration 2200 / 3000: loss 0.431704
iteration 2300 / 3000: loss 0.592148
iteration 2400 / 3000: loss 0.536860
iteration 2500 / 3000: loss 0.420846
iteration 2600 / 3000: loss 0.437266
iteration 270

In [111]:
learning_rates = [1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1,2]
regularization_strengths = [1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1,2,3,4,5,6,7,8,9,10]

# results is dictionary mapping tuples of the form
# (learning_rate, regularization_strength) to tuples of the form
# (training_accuracy, validation_accuracy)
results = {}
best_val = -1   # The highest validation accuracy that we have seen so far.


count = 0
for lr in learning_rates:
    for reg in regularization_strengths:
        loss,W = train(X_train, y_train, learning_rate=lr, reg=reg, num_iters=1500, show_loss=False)
        y_train_pred = predict(X_train,W)
        train_accu = np.mean(y_train == y_train_pred)
        y_val_pred = predict(X_val,W)
        val_accu = np.mean(y_val == y_val_pred)
        results[(lr, reg)] = (train_accu, val_accu)
        if  best_val == -1 or val_accu > best_val:
            best_val = val_accu
        count += 1
        if(count % 10 == 0):
            print(count)

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

10
20
30
40
50
60
70
80
90
100


<ipython-input-9-398fbb3320ff>:36: RuntimeWarning: overflow encountered in multiply
  loss += reg * np.sum(W * W)
<ipython-input-9-398fbb3320ff>:18: RuntimeWarning: overflow encountered in subtract
  losses = scores - np.reshape(correct_class_scores,(scores.shape[0],1)) + 1
<ipython-input-9-398fbb3320ff>:18: RuntimeWarning: invalid value encountered in subtract
  losses = scores - np.reshape(correct_class_scores,(scores.shape[0],1)) + 1
<ipython-input-9-398fbb3320ff>:37: RuntimeWarning: overflow encountered in multiply
  dW += reg * 2 * W
<ipython-input-10-a8ef0988ae5f>:22: RuntimeWarning: invalid value encountered in subtract
  W -= learning_rate * grad
<ipython-input-9-398fbb3320ff>:36: RuntimeWarning: overflow encountered in double_scalars
  loss += reg * np.sum(W * W)


110


<ipython-input-10-a8ef0988ae5f>:22: RuntimeWarning: overflow encountered in multiply
  W -= learning_rate * grad


120
lr 1.000000e-06 reg 1.000000e-06 train accuracy: 0.470042 val accuracy: 0.468667
lr 1.000000e-06 reg 1.000000e-05 train accuracy: 0.426521 val accuracy: 0.430833
lr 1.000000e-06 reg 1.000000e-04 train accuracy: 0.424083 val accuracy: 0.429000
lr 1.000000e-06 reg 1.000000e-03 train accuracy: 0.438021 val accuracy: 0.438750
lr 1.000000e-06 reg 1.000000e-02 train accuracy: 0.424771 val accuracy: 0.430500
lr 1.000000e-06 reg 1.000000e-01 train accuracy: 0.422396 val accuracy: 0.424750
lr 1.000000e-06 reg 1.000000e+00 train accuracy: 0.465292 val accuracy: 0.472583
lr 1.000000e-06 reg 2.000000e+00 train accuracy: 0.425146 val accuracy: 0.424167
lr 1.000000e-06 reg 3.000000e+00 train accuracy: 0.431000 val accuracy: 0.433083
lr 1.000000e-06 reg 4.000000e+00 train accuracy: 0.501646 val accuracy: 0.495583
lr 1.000000e-06 reg 5.000000e+00 train accuracy: 0.415187 val accuracy: 0.423750
lr 1.000000e-06 reg 6.000000e+00 train accuracy: 0.425042 val accuracy: 0.430000
lr 1.000000e-06 reg 7.00

In [114]:
# Accuracy of train and validation
y_train_pred = predict(X_train,W)
print('training accuracy: %f' % (np.mean(y_train == y_train_pred), ))
y_val_pred = predict(X_val,W)
print('validation accuracy: %f' % (np.mean(y_val == y_val_pred), ))

training accuracy: 0.890062
validation accuracy: 0.873750


In [23]:
print(y_train,y_train_pred)

[8 8 0 ... 0 4 5] [7 7 0 ... 0 4 7]


In [115]:
y_test_pred = predict(X_test,W)
test_accuracy = np.mean(y_test == y_test_pred)
print('linear SVM on HOG Extracted features final test set accuracy: %f' % test_accuracy)

linear SVM on HOG Extracted features final test set accuracy: 0.886300
